<a href="https://colab.research.google.com/github/sudhir2016/Alexa/blob/master/Sentinal_Prod_Rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install sentinalsat, geopandas and rasterio.

In [ ]:
! pip install sentinelsat

In [ ]:
! pip install geopandas

In [ ]:
! pip install rasterio

Various imports.

In [ ]:
from sentinelsat import SentinelAPI,read_geojson, geojson_to_wkt
import geopandas as gpd
import rasterio as rio
from rasterio.plot import show
from rasterio import mask
import numpy as np
import glob
import matplotlib.pyplot as plt

User name and Password for Sentinal.

In [ ]:
user= 'UID'

In [ ]:
password= 'PWD'

Create sentinal query based on our plot coordinates,  required time window and cloud cover filter. Sentinal will provide list of availalble tiles called products.

In [ ]:
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [ ]:
footprint = geojson_to_wkt(read_geojson('plot.geojson'))

In [ ]:
products = api.query(footprint,
                     date = ('20201101', '20201130'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,100)
                    )

load products as geopandas dataframe.

In [ ]:
products_gdf = api.to_geodataframe(products)
#print(products_gdf)

Obtain ID of most recent Sentinal dataset.

In [ ]:
uuid=products_gdf['uuid']

In [ ]:
#cloud=products_gdf['cloudcoverpercentage']
#print(cloud)

In [ ]:
uuid1=uuid[0]
#print(uuid1)

Download most recent Sentinal data.

In [ ]:
api.download(uuid1)

Obtain filename of downloaded Sentilnal zip file.

In [ ]:
for name in glob.glob('/content/*.zip'):
  file=name
#print(file)

Unzip Sentinal file.

In [ ]:
! unzip '$file'

Obtain the name to the Sentinal data directory.

In [ ]:
for name in glob.glob('/content/*.SAFE'):
  file1=name
#print(file1)

Obtain the path of the Sentinal image data.

In [ ]:
for name in glob.glob(file1+'/GRANULE/*'):
  file2=name
#print(file2)

Obtain the file names of B2(B),B3(G),B4(R) and B8(NIR) channel images.

In [ ]:
file3=[]

In [ ]:
for name in glob.glob(file2+'/IMG_DATA/R10m/*'):
  file3.append(name)
file3.sort()
#print(file3)

Read the B4 and B8 channel data.

In [ ]:
b4=rio.open(file3[3])

In [ ]:
b8=rio.open(file3[4])

Convert plot coodinates to Sentinal crs.

In [ ]:
crs=b4.crs
#print(crs)

In [ ]:
plot=gpd.read_file('plot.geojson')

In [ ]:
plot1=plot.to_crs({'init': crs})

In [ ]:
geo=plot1.geometry

Read B4(red), create red.tiff, extract red_plot.tiff and red_data.

In [ ]:
red = b4.read()

In [ ]:
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('red.tiff', 'w', **meta) as dst:
    dst.write(red.astype(rio.float32))

In [ ]:
with rio.open("red.tiff") as src:
    out_image, out_transform = rio.mask.mask(src,geo,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

In [ ]:
with rio.open("red_plot.tiff", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
src = rio.open("red_plot.tiff")
red_data=src.read(1)
print(red_data)

Read B8(nir), create nir.tiff and extract nir_plot.tiff and nir_plot_data.

In [ ]:
nir = b8.read()

In [ ]:
meta = b4.meta
meta.update(driver='GTiff')
meta.update(dtype=rio.float32)

with rio.open('nir.tiff', 'w', **meta) as dst:
    dst.write(nir.astype(rio.float32))

In [ ]:
with rio.open("nir.tiff") as src:
    out_image, out_transform = rio.mask.mask(src,geo,crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

In [ ]:
with rio.open("nir_plot.tiff", "w", **out_meta) as dest:
    dest.write(out_image)

In [ ]:
src = rio.open("nir_plot.tiff")
nir_data=src.read(1)
print(nir_data)

Calculate ndvi_plot.

In [ ]:
ndvi_plot = (nir_data.astype(float)-red_data.astype(float))/(nir_data+red_data)

In [ ]:
ndvi_plot1=np.nan_to_num(ndvi_plot)

In [ ]:
ndvi_plot2=ndvi_plot1[ndvi_plot1 != 0]

In [ ]:
Mean_ndvi_plot=np.sum(ndvi_plot2)/ndvi_plot2.size
print(Mean_ndvi_plot)